# PB016: Artificial Intelligence I, lab 4 - Problem decomposition, CSP

This week's topic are decomposition and various sample representations of problems, as well as their solution using search and constraint satisfaction. We'll focus namely on:

1. __Decomposition of problems using AND/OR trees__
2. __Constraint satisfaction problems__

---

## 1. Decomposition of problems using [AND/OR trees](https://en.wikipedia.org/wiki/And%E2%80%93or_tree)

__Basic facts__
- AND / OR trees are graphical representations of a problem and its recursive reduction to conjunctions and disjunctions of subproblems.
- There are two alternating types of nodes in the tree - AND and OR.
  - An AND node has a solution if all its successor nodes (neighbours) have a solution. The edges between the AND node and its successors are connected by an arc according to the standard notation convention, which distinguishes them from the OR nodes.
  - An OR node has a solution if at least one of its successor nodes has a solution.
- Leaves are atomic problems for which we know whether they have a solution or not.
- The problem represented this way can be solved by searching the graph (using DFS, BFS, best-first search, etc.).

__Example__ - a sample abstract AND/OR tree:

![AND/OR tree](https://www.fi.muni.cz/~novacek/courses/pb016/labs/img/andortree.png)

 ### __Exercise 1.1: Representation of a dragon problem using an AND/OR tree__
- Use the [NetworkX](https://networkx.github.io/) library (or any other approach that suits you) to create an AND/OR tree representing the following problem:
 - The primary goal is to _free a kingdom from the rule of a dragon-tyrant_. This can be solved either by _confronting the tyrant_ or by _praying that the dragon will get the hell out of its den and fly where it came from_.
 - In order to _stand up to the dragon_, we must _go to his den_ and confront him.
 - The confrontation with the dragon itself can then be solved in various ways: we can _try to use our strength and kill the dragon_, or we can _rely on our eloquence and convince the dragon that being a tyrant is not OK_.
- Hints:
 - Without loss of generality, it can be assumed that the root node is an OR one, and in each subsequent level of the tree the types of nodes alternate regularly (therefore it's not necessary to explicitly store this information in the graph).
 - To describe the (sub)problems (i.e. graph nodes) and store information about their being solved or not, you can use the corresponding global annotation dictionaries `description` and` solved`. Alternatively, these can be passed as arguments to your functions.
 - At the beginning, mark the leaf problems that could be atomic solutions as solved and consider the rest unresolved. The actual atomic solutions are entirely up to you (note that the sample solution is nevertheless pacifist).

In [2]:
import networkx as nx
from typing import Hashable

# representation of the AND/OR tree by an oriented graph
G = nx.DiGraph()

# filling the graph with nodes representing the problem and its decomposition
# (e.g. using function G.add_nodes_from([1,2,3,...]))
def add_node(from_node: Hashable, to_node: Hashable, node_type: str):
    if to_node in G:
        assert G[to_node].node_type == node_type
    else:
        G.add_node(to_node, node_type=node_type)
    if from_node is not None:
        G.add_edge(from_node, to_node)

def add_and_node(from_node: Hashable, to_node: Hashable):
    add_node(from_node, to_node, "AND")

def add_or_node(from_node: Hashable, to_node: Hashable):
    add_node(from_node, to_node, "OR")

# AND node without children is equal to True
# OR node without children is equal to False
add_or_node(None, "Free")
add_and_node("Free", "Stand up")
add_and_node("Stand up", "Go to den")
add_or_node("Stand up", "Confront")
add_and_node("Confront", "Kill")
add_and_node("Confront", "Convince")
add_and_node("Free", "Pray")

# initialize the appropriate dictionaries with node annotations

description = {
    "Free": "Free a kingdom from the rule of a dragon-tyrant",
    "Stand up": "Stand up the dragon",
    "Pray": "Pray that the dragon will get the hell out of its den and fly where it came from",
    "Go to den": "Go to dragon's den",
    "Confront": "Confront the dragon",
    "Kill": "Try to use our strength and kill the dragon",
    "Convince": "Rely on our eloquence and convince the dragon that being a tyrant is not OK"
}

solved = dict()
"""
solved = {
    "Free": False,
    "Stand up": False,
    "Pray": True,
    "Go to den": False,
    "Confront": False,
    "Kill": True,
    "Convince": True
}
"""

'\nsolved = {\n    "Free": False,\n    "Stand up": False,\n    "Pray": True,\n    "Go to den": False,\n    "Confront": False,\n    "Kill": True,\n    "Convince": True\n}\n'

### Main function for searching the AND/OR tree
- Depth-search of the created AND/OR tree, checking for a solution of the dragon problem.
- Recursively searches the problem representation tree from the primary goal node (i.e. the root node), calling specific functions for AND and OR nodes and continuously updating the `solved` values for each node.

In [3]:
def expand_node(graph, problem):
    for neighbor in graph[problem]:
        and_or_search(graph, neighbor)

def and_or_search(graph, problem):
    expand_node(graph, problem)
    
    problem_type = graph.nodes[problem]["node_type"]
    if problem_type == "AND":
        solved[problem] = True
        for neighbor in graph[problem]:
            if not solved[neighbor]:
                solved[problem] = False
    elif problem_type == "OR":
        solved[problem] = False
        for neighbor in graph[problem]:
            if solved[neighbor]:
                solved[problem] = True
    else:
        # Node with other type
        assert False

In [4]:
def and_or_depth_first_search(graph, problem):
    """A wrapper function to perform the AND/OR tree search. The nodes
    alternate, starting with an OR node.

    NOTE: The function does not return anything. Instead, it updates the GLOBAL
    structure `solved`, which can then be used for constructing the solution
    sub-tree of the original problem tree.

    Parameters
    ----------
    graph : networkx.Graph
        The AND-OR graph representing the top-level problem and its subproblems.
    problem : int
        The node in the graph representing the current (sub)problem to be
        solved.
    """

    and_or_search(graph, problem)

### __Exercise 1.2: Solving the dragon problem by depth-searching the AND/OR tree - OR nodes__
- Implement the function for exploring the OR nodes.
- Hint - calls the function for searching AND nodes for all descendants of the node.

In [5]:
def or_search(graph, problem):
    """Realises the OR part of the AND/OR tree search.

    NOTE: The function does not return anything. Instead, it updates the GLOBAL
    structure `solved`, which can then be used for constructing the solution
    sub-tree of the original problem tree.

    Parameters
    ----------
    graph : networkx.Graph
        The AND-OR graph representing the top-level problem and its subproblems.
    problem : int
        The node in the graph representing the current (sub)problem to be
        solved.
    """

    and_or_search(graph, problem)

### __Exercise 1.3: Solving the dragon problem by depth-searching the AND/OR tree - AND nodes__
- Implement the function for exploring the AND nodes.
- Hint - calls the function for searching OR nodes for all descendants of the node.

In [6]:
# searching the AND nodes

def and_search(graph, problem):
    """Realises the AND part of the AND/OR tree search.

    NOTE: The function does not return anything. Instead, it updates the GLOBAL
    structure `solved`, which can then be used for constructing the solution
    sub-tree of the original problem tree.

    Parameters
    ----------
    graph : networkx.Graph
        The AND-OR graph representing the top-level problem and its subproblems.
    problem : int
        The node in the graph representing the current (sub)problem to be
        solved.
    """

    and_or_search(graph, problem)

### Testing the solution existence

In [7]:
# testing the existence of the solution
and_or_depth_first_search(G, "Free")
if solved["Free"]:
    print('Solution exists.')
else:
    print('Solution does not exist.')

Solution exists.


### __Exercise 1.4: Listing the solution plan__
- Search the solved tree and print out the decomposition of the main problem into subproblems that have solutions.
- Hint:
 - A possible way how to implement this is similar to the search algorithm itself.
 - One may recursively call the composition of the plan text using alternate traversal of AND and OR nodes, keeping track of which node one is in (e.g., via a map of dual nodes, as in the sample solution).
 - The plan text itself may follow for instance the form: $G \; \leftarrow \; S_1 \; [AND | OR] \; S_2 \; [AND | OR] \; \dots \; [AND | OR] \; S_n $ for problem $ G $ and subproblems $ S_1, \dots, S_n $.
 - An example (multiple) solution plan: _Get rid of the dragon-tyrant. <-- (Stand up to the dragon. <-- (Go to the dragon's den. <-- (Convince the dragon not to be a tyrant.)) OR Pray for the dragon to disappear.)_

In [8]:
def generate_plan(graph, problem):
    """A wrapper function for generating a solution plan by calling the
    `generate_sub_plan()` function for the top-level (OR) node.

    Parameters
    ----------
    graph : networkx.Graph
        The AND-OR graph representing the top-level problem and its subproblems.
    problem : int
        The node in the graph representing the current (sub)problem to be
        solved.

    Returns
    -------
    str
        The string representation of the solution plan based on the
        solution sub-tree of the original AND-OR problem tree that is
        determined by the `True` node values recorded in the `solved` global
        structure.
    """

    return generate_sub_plan(graph, problem)


def generate_sub_plan(graph, problem):
    """Generate sub-plan for OR or AND node (to be called in an alternating
    manner).

    Parameters
    ----------
    graph : networkx.Graph
        The AND-OR graph representing the top-level problem and its subproblems.
    problem : int
        The node in the graph representing the current (sub)problem to be
        used for generating the solution plan.

    Returns
    -------
    str
        The string representation of the plan based on the solution of the
        current sub-problem.
    """

    print(f"generating for: {problem}")
    if not graph[problem]:
        # No children -> leaf node
        return f" {problem} "
    return f" {problem} <- ({graph.nodes[problem]['node_type'].join(map(lambda x: generate_sub_plan(graph, x), graph[problem]))}) "

In [9]:
# print a plan corresponding to the solution of the previous example
if solved["Free"]:
    print('Solution exists and the corresponding plan is: \n')
    print(generate_plan(G, "Free"))
else:
    print('Solution does not exist.')

Solution exists and the corresponding plan is: 

generating for: Free
generating for: Stand up
generating for: Go to den
generating for: Confront
generating for: Kill
generating for: Convince
generating for: Pray
 Free <- ( Stand up <- ( Go to den AND Confront <- ( Kill OR Convince ) ) OR Pray ) 


#### __Food for thought__
- How could the AND/OR tree representation be modified to use a best-first search?
- Hint:
  - Annotation of nodes, or edges with weights that reflect preferred solutions to subproblems.
  - Greedy unpacking of nodes connected to the parent via an edge with a better weight.

---

## 2. Constraint satisfaction problems ([CSP](https://en.wikipedia.org/wiki/Constraint_satisfaction_problem))

__Basic facts__
- Modeling problems as sets of variables that can take values from predefined domains.
- The values of the variables are limited by a set of constraints that correspond to the structure of the problem.
- The solution of the problem then corresponds to finding such an assignment of values to individual variables that it satisfies all the constraints and doesn't omit any variable.
- The field is an intensive subject of research in informatics and especially AI, and thus there are a number of off-the-shelf tools to address CSP.

### Example of a simple abstract CSP problem and its solution
- Problem with two variables $A \in \{1,2,3\} $ and $B \in \{4,5,6\}$ and one constraint $2A = B$.
- Representation of variables using the package [`python-constraint`](https://github.com/python-constraint/python-constraint):

In [10]:
# installing the package first
!python3.11 -m pip install python-constraint

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [11]:
# defining the problem variables
from constraint import *

problem = Problem()
problem.addVariable('A', [1, 2, 3])
problem.addVariable('B', [4, 5, 6])

- Solving the problem without the constraint:

In [12]:
problem.getSolutions()

[{'A': 3, 'B': 6},
 {'A': 3, 'B': 5},
 {'A': 3, 'B': 4},
 {'A': 2, 'B': 6},
 {'A': 2, 'B': 5},
 {'A': 2, 'B': 4},
 {'A': 1, 'B': 6},
 {'A': 1, 'B': 5},
 {'A': 1, 'B': 4}]

- Solving the problem with the constraint:

In [13]:
# here we add a constraint as a lambda function, which is applied to vars A, B
problem.addConstraint(lambda x, y: 2 * x == y, ('A','B'))
problem.getSolutions()

[{'A': 3, 'B': 6}, {'A': 2, 'B': 4}]

- Solving the problem with another variable and constraint:

In [14]:
problem.addVariable('C', [1, 2])

# again, constraint is given as a lambda function, this time applied to var C
problem.addConstraint(lambda x: x == 1, ('C'))
problem.getSolutions()

[{'A': 3, 'B': 6, 'C': 1}, {'A': 2, 'B': 4, 'C': 1}]

### __Exercise 2.1: Map coloring problem__
- Let us consider the following map of Australia from the problem discussed in the lecture:

![Australia](https://www.fi.muni.cz/~novacek/courses/pb016/labs/img/australia.png)

- Use the CSP library [`python-constraint`](https://github.com/python-constraint/python-constraint) to solve the problem of coloring this map with three colors so that no two neighboring states have the same color.

In [15]:
# initialising the problem
map_colouring = Problem()

# adding variables (states), each with the same colour domain
map_colouring.addVariable('WA', ['red', 'blue', 'green'])  # Western Australia
map_colouring.addVariable('NT', ['red', 'blue', 'green'])  # Northern Territory
map_colouring.addVariable('Q', ['red', 'blue', 'green'])   # Queensland
map_colouring.addVariable('NSW', ['red', 'blue', 'green']) # New South Wales
map_colouring.addVariable('V', ['red', 'blue', 'green'])   # Victoria
map_colouring.addVariable('SA', ['red', 'blue', 'green'])  # South Australia
map_colouring.addVariable('T', ['red', 'blue', 'green'])   # Tasmania

# adding contraints using map_colouring.addConstraint(...)

neighbors = [
    ('WA', 'NT'),
    ('WA', 'SA'),
    ('NT', 'SA'),
    ('NT', 'Q'),
    ('SA', 'Q'),
    ('SA', 'NSW'),
    ('SA', 'V'),
    ('Q', 'NSW'),
    ('NSW', 'V')
]

constraint_func = (lambda x, y: x != y)

for teritories in neighbors:
    map_colouring.addConstraint(constraint_func, teritories)

# computing and printing the result
# NOTE: do not run before adding some constraints (too many solutions printed)
solutions = map_colouring.getSolutions()
print(f'Found {len(solutions)} solutions for colouring the Australia map:')
print('\n'.join([str(x) for x in solutions]))

Found 18 solutions for colouring the Australia map:
{'SA': 'green', 'NSW': 'blue', 'Q': 'red', 'NT': 'blue', 'V': 'red', 'WA': 'red', 'T': 'green'}
{'SA': 'green', 'NSW': 'blue', 'Q': 'red', 'NT': 'blue', 'V': 'red', 'WA': 'red', 'T': 'blue'}
{'SA': 'green', 'NSW': 'blue', 'Q': 'red', 'NT': 'blue', 'V': 'red', 'WA': 'red', 'T': 'red'}
{'SA': 'green', 'NSW': 'red', 'Q': 'blue', 'NT': 'red', 'V': 'blue', 'WA': 'blue', 'T': 'green'}
{'SA': 'green', 'NSW': 'red', 'Q': 'blue', 'NT': 'red', 'V': 'blue', 'WA': 'blue', 'T': 'blue'}
{'SA': 'green', 'NSW': 'red', 'Q': 'blue', 'NT': 'red', 'V': 'blue', 'WA': 'blue', 'T': 'red'}
{'SA': 'blue', 'NSW': 'green', 'Q': 'red', 'NT': 'green', 'V': 'red', 'WA': 'red', 'T': 'green'}
{'SA': 'blue', 'NSW': 'green', 'Q': 'red', 'NT': 'green', 'V': 'red', 'WA': 'red', 'T': 'blue'}
{'SA': 'blue', 'NSW': 'green', 'Q': 'red', 'NT': 'green', 'V': 'red', 'WA': 'red', 'T': 'red'}
{'SA': 'blue', 'NSW': 'red', 'Q': 'green', 'NT': 'red', 'V': 'green', 'WA': 'green', 'T

 ### __*Exercise 2.2: Representation of the [8 queens puzzle](https://en.wikipedia.org/wiki/Eight_queens_puzzle) as CSP__
- Represent the 8 queens problem using the CSP library [`python-constraint`](https://github.com/python-constraint/python-constraint).
- Notes on the solution:
 - For more concise code, you can use the `addVariables(variables,domain)` function. It bulk-adds `variables`, each of which can take one of the values ​​in the `domain` set.
 - Variables can correspond, for example, to the columns where the individual queens are located on the board (we know that there can be a maximum of one queen in one column).
 - The values ​​of the variables then correspond to the rows in which the queens are located.
 - Example: If the variables correspond to numbers from $0$ to $7$ (i.e. Python indexes of the column coordinates on the board) and the possible values ​​of these variables also have a domain of numbers from $0$ to $7$, assigning the value of $1$ to the variable $7$ corresponds to placing the queen in the second row in the last column.

In [16]:
# initialising the problem and the board size
eight_queens = Problem()
board_size = 8

# Each queen is in hers own column
# Already constraints solution
eight_queens.addVariables([f"c{col}" for col in range(board_size)], range(board_size))

count = 0
for sol in eight_queens.getSolutionIter():
    print(sol)
    count += 1
    if count > 10:
        break

{'c0': 7, 'c1': 7, 'c2': 7, 'c3': 7, 'c4': 7, 'c5': 7, 'c6': 7, 'c7': 7}
{'c0': 7, 'c1': 7, 'c2': 7, 'c3': 7, 'c4': 7, 'c5': 7, 'c6': 7, 'c7': 6}
{'c0': 7, 'c1': 7, 'c2': 7, 'c3': 7, 'c4': 7, 'c5': 7, 'c6': 7, 'c7': 5}
{'c0': 7, 'c1': 7, 'c2': 7, 'c3': 7, 'c4': 7, 'c5': 7, 'c6': 7, 'c7': 4}
{'c0': 7, 'c1': 7, 'c2': 7, 'c3': 7, 'c4': 7, 'c5': 7, 'c6': 7, 'c7': 3}
{'c0': 7, 'c1': 7, 'c2': 7, 'c3': 7, 'c4': 7, 'c5': 7, 'c6': 7, 'c7': 2}
{'c0': 7, 'c1': 7, 'c2': 7, 'c3': 7, 'c4': 7, 'c5': 7, 'c6': 7, 'c7': 1}
{'c0': 7, 'c1': 7, 'c2': 7, 'c3': 7, 'c4': 7, 'c5': 7, 'c6': 7, 'c7': 0}
{'c0': 7, 'c1': 7, 'c2': 7, 'c3': 7, 'c4': 7, 'c5': 7, 'c6': 6, 'c7': 7}
{'c0': 7, 'c1': 7, 'c2': 7, 'c3': 7, 'c4': 7, 'c5': 7, 'c6': 6, 'c7': 6}
{'c0': 7, 'c1': 7, 'c2': 7, 'c3': 7, 'c4': 7, 'c5': 7, 'c6': 6, 'c7': 5}


### __*Exercise 2.3: Solving 8 queens as CSP__
- Solve the 8 queens problem using the CSP library [`python-constraint`](https://github.com/python-constraint/python-constraint).
- Hint - the solution is very similar to the pseudocode from the lecture. In particular, it's quite handy to generate the constraints in a nested cycle that corresponds to a chosen representation of the placement of queens - for instance, "from left to right".
- Each queen determines the constraints for queens "to the right" of her in this case (the first placed one for the next seven, the second for the next six, etc.).
- The constraints are the same as in the labs for the second week of the course, only they can be expressed rather more concisely (using a `lambda` function in the `addConstraint()` method). Note that a solution with a named, externally defined function is also perfectly fine, though (the `python-constraint` package supports any function as the argument of the `addConstraint()` method).

In [35]:
# traversing one queen at a time and adding conditions for the queens'
# position "to the right" of her

lambdaFactory = lambda par: (lambda x, y: x - y != par)

# Constraining only tuples of two different queens
for colA in range(board_size):
    for colB in range(colA):
        ca = int(colA)
        cb = int(colB)
        queens = (f"c{colA}", f"c{colB}")
        # Two queens do not share row
        eight_queens.addConstraint(lambda x, y: x != y, queens)
        # Two queens do not attack each other diagonally
        eight_queens.addConstraint(lambdaFactory(colA-colB), queens)
        eight_queens.addConstraint(lambdaFactory(colB-colA), queens)
        # eight_queens.addConstraint(lambda x, y: x - colA == y - colB, queens)
        # eight_queens.addConstraint(lambda x, y: x + colA != y + colB, queens)

# timed calculation of solutions
# NOTE: do not run before adding constraints, you'll probably run out of RAM
%time solutions = eight_queens.getSolutions()


def solution_parse(solution, n=8):
    # Parses solution from [q0, ..., q7] to [q0(x, y), ..., q7(x, y)]
    sol = []
    for key, value in solution.items():
        sol.append((int(key[1:]), value))
    return sol


def print_solution(solution, n=8):
    """Auxiliary function for printing the solutions.

    Parameters
    ----------
    solution : dct
        A mapping from variables names to their values corresponding to a
        solution of the CSP.
    n : 8, optional (default is 8) for the 8-queens problem

    Returns
    -------
    str
        A string representation of the solution board.
    """

    rows, positions = [], solution
    for i in range(n):
        column = []
        for j in range(n):
            if (i, j) in positions:
                column.append('Q')
            else:
                column.append('-')
        rows.append(' '.join(column))
    return '\n'.join(rows)


# printing the solutions
print('Number of solutions:', len(solutions))
print('List of solutions:\n'+'\n\n'.join(['Solution '+str(i+1)+':\n'+ \
                                          print_solution(solution_parse(solutions[i])) \
                                          for i in range(len(solutions))]))

CPU times: user 1.05 s, sys: 0 ns, total: 1.05 s
Wall time: 1.05 s
Number of solutions: 92
List of solutions:
Solution 1:
- - - - - - - Q
- Q - - - - - -
- - - - Q - - -
- - Q - - - - -
Q - - - - - - -
- - - - - - Q -
- - - Q - - - -
- - - - - Q - -

Solution 2:
- - - - - - - Q
- Q - - - - - -
- - - Q - - - -
Q - - - - - - -
- - - - - - Q -
- - - - Q - - -
- - Q - - - - -
- - - - - Q - -

Solution 3:
- - - - - - - Q
- - Q - - - - -
Q - - - - - - -
- - - - - Q - -
- Q - - - - - -
- - - - Q - - -
- - - - - - Q -
- - - Q - - - -

Solution 4:
- - - - - - - Q
- - - Q - - - -
Q - - - - - - -
- - Q - - - - -
- - - - - Q - -
- Q - - - - - -
- - - - - - Q -
- - - - Q - - -

Solution 5:
- - - - - - Q -
- Q - - - - - -
- - - - - Q - -
- - Q - - - - -
Q - - - - - - -
- - - Q - - - -
- - - - - - - Q
- - - - Q - - -

Solution 6:
- - - - - - Q -
- Q - - - - - -
- - - Q - - - -
Q - - - - - - -
- - - - - - - Q
- - - - Q - - -
- - Q - - - - -
- - - - - Q - -

Solution 7:
- - - - - - Q -
Q - - - - - - -


 ---

#### _Final note_ - the materials used in this notebook are original works adapted from the original authors as follows:
- Example of an AND/OR tree:
  - Retrieved from [ResearchGate](https://www.researchgate.net/profile/Mark_Winands/publication/259655486/figure/fig1/AS:297012311412737@1447824661825/Example-of-an-AND-OR-tree-OR-nodes-shown-as-squares-AND-nodes-as-circles.png)
  - Author: Kishimoto, Akihiro et al.
  - Original source: Kishimoto, Akihiro & Winands, Mark & Müller, Martin & Saito, Jahn-Takeshi. (2012). Game-Tree Search Using Proof Numbers: The First Twenty Years. ICGA journal. 35. 131-156. 10.3233/ICG-2012-35302.
  - License: N/A